In [1]:
%reload_ext autoreload
%autoreload 2
# %matplotlib qt

from time import time
from tqdm import tqdm
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

from cellector.io import create_from_suite2p
from cellector.gui import SelectionGUI

In [21]:
# Next step: 
# 2. Visualization methods for optimizing the hyperparameters for preprocessing
# 3. Implement an optimization method for phase_corr_eps? 
#    - Might just be as simple as the user inputing the maximum size ROI, and then measuring
#      the average power for higher frequencies...

In [2]:
s2p_dir = Path(r"C:\Users\Andrew\Documents\test_cellector\suite2p")
roi_processor = create_from_suite2p(s2p_dir, use_redcell=True, clear_existing=False)

In [20]:
gui = SelectionGUI(roi_processor)

In [ ]:


# # Hyperparameter Choices:
# So far the only hyperparameters I'm aware of are filtering parameters, and the eps value for
# phase correlation measurements (which is weirdly high...?). I think it would be good to do 
# some hyperparameter optimization for these, which a user could manually supervise themselves
# with some labelling. For example, the user could open a GUI that compares masks with reference
# images for some sample "true" data and in addition for any data they've loaded in. The GUI might
# look something like follows:
# 1. For a particular set of hyperparameters (filtering, for example), the user could get a histogram
# of feature values for all the features for all the masks. They could use cutoff lines to pick a range
# of feature values for that particular set of hyperparameters, and then scroll through mask matches
# that come from within that range. This way, they could determine how the hyperparameters affect the
# feature values at each part of the distribution and select hyperparameters that give good separation.
